In [24]:
# import libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
import lightgbm as lgb 
from sklearn.metrics import mean_absolute_error

In [25]:
# import data

prop = pd.read_csv('C:\\Users\\PC\Documents\\Python Scripts\\Zillow\\properties_2016.csv',\
                  low_memory=False)
train = pd.read_csv('C:\\Users\\PC\Documents\\Python Scripts\\Zillow\\train_2016.csv',\
                  low_memory=False)
sample = pd.read_csv('C:\\Users\\PC\Documents\\Python Scripts\\Zillow\\sample_submission.csv',\
                  low_memory=False)


In [26]:
# merge dataframes

df_train = train.merge(prop, how='left', on='parcelid')

In [27]:
median_values = df_train.median(axis=0)
df_train_new = df_train.fillna(median_values, inplace=True)

In [28]:
df_train['transactiondate'] = pd.to_datetime(df_train.transactiondate)
df_train_new['transactiondate'] = pd.to_numeric(df_train_new['transactiondate'])

In [29]:
#Encode columns containing object data types

from sklearn import preprocessing 

le = preprocessing.LabelEncoder()

df_train_new['hashottuborspa'] = le.fit_transform(df_train_new['hashottuborspa'])
df_train_new['fireplaceflag'] = le.fit_transform(df_train_new['fireplaceflag'])

df_train_new['propertyzoningdesc'] = df_train_new['propertyzoningdesc'].factorize()[0]
df_train_new['propertyzoningdesc'] = le.fit_transform(df_train_new['propertyzoningdesc'])

df_train_new['propertycountylandusecode'] = df_train_new['propertycountylandusecode'].factorize()[0]
df_train_new['propertycountylandusecode'] = le.fit_transform(df_train_new['propertycountylandusecode'])

df_train_new['taxdelinquencyflag'] = df_train_new['taxdelinquencyflag'].factorize()[0]
df_train_new['taxdelinquencyflag'] = le.fit_transform(df_train_new['taxdelinquencyflag'])

C:\Users\PC\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [30]:
#df_train_new.info()
#df_train_new.drop('transactiondate', axis=1, inplace=True)
df_train_new.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,1451606400000000000,1.0,7.0,616.0,2.0,3.0,4.0,4.0,...,1.0,0,122754.0,360170.0,2015.0,237416.0,6735.88,0,14.0,6.037107e+13
1,14366692,-0.1684,1451606400000000000,1.0,7.0,616.0,3.5,4.0,4.0,7.0,...,1.0,286,346458.0,585529.0,2015.0,239071.0,10153.02,0,14.0,6.037615e+13
2,12098116,-0.0040,1451606400000000000,1.0,7.0,616.0,3.0,2.0,4.0,4.0,...,1.0,286,61994.0,119906.0,2015.0,57912.0,11484.48,0,14.0,6.037464e+13
3,12643413,0.0218,1451692800000000000,1.0,7.0,616.0,2.0,2.0,4.0,4.0,...,1.0,286,171518.0,244880.0,2015.0,73362.0,3048.74,0,14.0,6.037296e+13
4,14432541,-0.0050,1451692800000000000,1.0,7.0,616.0,2.5,4.0,4.0,7.0,...,2.0,286,169574.0,434551.0,2015.0,264977.0,5488.96,0,14.0,6.059042e+13


In [31]:
# define predictors (X) and target (y)

#y = pd.DataFrame(data['logerror'])
y=df_train_new['logerror'].ravel()

In [32]:
X = df_train_new.drop('logerror', axis=1, inplace=True)
X=df_train_new.drop('transactiondate', axis=1, inplace=True)
X = df_train_new
X.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,1.0,7.0,616.0,2.0,3.0,4.0,4.0,2.0,66.0,...,1.0,0,122754.0,360170.0,2015.0,237416.0,6735.88,0,14.0,6.037107e+13
1,14366692,1.0,7.0,616.0,3.5,4.0,4.0,7.0,3.5,66.0,...,1.0,286,346458.0,585529.0,2015.0,239071.0,10153.02,0,14.0,6.037615e+13
2,12098116,1.0,7.0,616.0,3.0,2.0,4.0,4.0,3.0,66.0,...,1.0,286,61994.0,119906.0,2015.0,57912.0,11484.48,0,14.0,6.037464e+13
3,12643413,1.0,7.0,616.0,2.0,2.0,4.0,4.0,2.0,66.0,...,1.0,286,171518.0,244880.0,2015.0,73362.0,3048.74,0,14.0,6.037296e+13
4,14432541,1.0,7.0,616.0,2.5,4.0,4.0,7.0,2.5,66.0,...,2.0,286,169574.0,434551.0,2015.0,264977.0,5488.96,0,14.0,6.059042e+13


In [33]:
# split dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [34]:
#Encode columns containing object data types

from sklearn import preprocessing 

le = preprocessing.LabelEncoder()

prop['hashottuborspa'] = le.fit_transform(prop['hashottuborspa'])
prop['fireplaceflag'] = le.fit_transform(prop['fireplaceflag'])

prop['propertyzoningdesc'] = prop['propertyzoningdesc'].factorize()[0]
prop['propertyzoningdesc'] = le.fit_transform(prop['propertyzoningdesc'])

prop['propertycountylandusecode'] = prop['propertycountylandusecode'].factorize()[0]
prop['propertycountylandusecode'] = le.fit_transform(prop['propertycountylandusecode'])

prop['taxdelinquencyflag'] = prop['taxdelinquencyflag'].factorize()[0]
prop['taxdelinquencyflag'] = le.fit_transform(prop['taxdelinquencyflag'])

C:\Users\PC\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [35]:
median_values = prop.median(axis=0)
prop = prop.fillna(median_values, inplace=True)
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,1.0,7.0,534.0,0.0,0.0,4.0,7.0,2.0,66.0,...,1.0,0,122590.0,9.0,2015.0,9.0,3991.78,0,14.0,6.037572e+13
1,10759547,1.0,7.0,534.0,0.0,0.0,4.0,7.0,2.0,66.0,...,1.0,6476,122590.0,27516.0,2015.0,27516.0,3991.78,0,14.0,6.037572e+13
2,10843547,1.0,7.0,534.0,0.0,0.0,4.0,7.0,2.0,66.0,...,1.0,6476,650756.0,1413387.0,2015.0,762631.0,20800.37,0,14.0,6.037572e+13
3,10859147,1.0,7.0,534.0,0.0,0.0,3.0,7.0,2.0,66.0,...,1.0,6476,571346.0,1156834.0,2015.0,585488.0,14557.57,0,14.0,6.037572e+13
4,10879947,1.0,7.0,534.0,0.0,0.0,4.0,7.0,2.0,66.0,...,1.0,6476,193796.0,433491.0,2015.0,239695.0,5725.17,0,14.0,6.037572e+13


In [36]:
#predict on columns contained in X

prop=prop[X.columns]


In [37]:
# Run XGBoost model on train & test sets

trn_x = X_train
trn_y = y_train
val_x = X_test
val_y = y_test

train_data = lgb.Dataset(trn_x, trn_y) # trn_x put the train dataset and its target
test_data = lgb.Dataset(val_x,val_y, reference=train_data) #val is the test set that you use

In [84]:
# setting up parameters - more details at https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.md

params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'mae',
    'sub_feature':0.6,
    'num_leaves':30,
    'min_data':250,
    'min_hessian':1,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_iterations':80, 
    'is_unbalanced':False,
    'early_stopping_round':5 # stops if no improvement after 5 iterations
}  

num_boost_round = 250 # number of iterations

lgb.cv(params, train_data, num_boost_round, nfold=5) # crossvalidates with 5 folds for selection
bst = lgb.train(params, train_data, num_boost_round, valid_sets=[test_data] )
print(bst.best_iteration)

[1]	valid_0's l1: 0.0686781
[2]	valid_0's l1: 0.0686688
[3]	valid_0's l1: 0.06866
[4]	valid_0's l1: 0.06865
[5]	valid_0's l1: 0.0686319
[6]	valid_0's l1: 0.0686262
[7]	valid_0's l1: 0.0686109
[8]	valid_0's l1: 0.0685974
[9]	valid_0's l1: 0.0685821
[10]	valid_0's l1: 0.0685693
[11]	valid_0's l1: 0.0685621
[12]	valid_0's l1: 0.0685477
[13]	valid_0's l1: 0.0685425
[14]	valid_0's l1: 0.0685266
[15]	valid_0's l1: 0.0685134
[16]	valid_0's l1: 0.0685005
[17]	valid_0's l1: 0.0684867
[18]	valid_0's l1: 0.0684739
[19]	valid_0's l1: 0.0684624
[20]	valid_0's l1: 0.0684568
[21]	valid_0's l1: 0.0684455
[22]	valid_0's l1: 0.0684386
[23]	valid_0's l1: 0.0684266
[24]	valid_0's l1: 0.0684145
[25]	valid_0's l1: 0.0684026
[26]	valid_0's l1: 0.0683919
[27]	valid_0's l1: 0.0683857
[28]	valid_0's l1: 0.0683742
[29]	valid_0's l1: 0.0683623
[30]	valid_0's l1: 0.068353
[31]	valid_0's l1: 0.0683436
[32]	valid_0's l1: 0.068335
[33]	valid_0's l1: 0.0683311
[34]	valid_0's l1: 0.0683218
[35]	valid_0's l1: 0.0683135


In [85]:
#Predict on prop file
pred = bst.predict(prop, num_iteration=bst.best_iteration)


In [86]:
print("Start writing to file ...")
for c in sample.columns[sample.columns != 'ParcelId']:
    sample[c] = pred


Start writing to file ...


In [87]:
sample.to_csv('submission.csv', index=False, float_format='%.4f')